# Week 3 Assignment
Segmenting and Clustering Neightbourhoods in Toronto

## Part 1: scrape the Wikipedia page

In [58]:
import requests
import pandas as pd
neighbourhood = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(neighbourhood, "html.parser")

table = soup.find("table")

#load the table header
for row in table.find_all("tr"):
    columns = row.find_all("th") 
    header = []
    for column in columns:
        header.append(column.text.replace("\n",""))
    break

#load the table values
Neighbourhood_table=[]
for row in table.find_all("tr"):
    columns = row.find_all("td") 
    output_row = []
    for column in columns:
        output_row.append(column.text.replace("\n",""))
    Neighbourhood_table.append(output_row)

#load the table into dataframe
Neighbourhood_table = pd.DataFrame(Neighbourhood_table[1:],columns=header) #skip the first row in Neighbourhood Table
print(Neighbourhood_table.head())

  Postal Code           Borough               Neighborhood
0         M1A      Not assigned                           
1         M2A      Not assigned                           
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


In [59]:
#filter out "Not assigned" Borough
Neighbourhood_table = Neighbourhood_table[Neighbourhood_table.Borough!="Not assigned"]
print(Neighbourhood_table.head(11))
print(Neighbourhood_table.shape)

   Postal Code           Borough                                 Neighborhood
2          M3A        North York                                    Parkwoods
3          M4A        North York                             Victoria Village
4          M5A  Downtown Toronto                    Regent Park, Harbourfront
5          M6A        North York             Lawrence Manor, Lawrence Heights
6          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
8          M9A         Etobicoke                             Islington Avenue
9          M1B       Scarborough                               Malvern, Rouge
11         M3B        North York                                    Don Mills
12         M4B         East York              Parkview Hill, Woodbine Gardens
13         M5B  Downtown Toronto                     Garden District, Ryerson
14         M6B        North York                                    Glencairn
(103, 3)


## Part 2: Tag Lat, Long to each neighbourhood

In [60]:
Geospatial_CSV = pd.read_csv("https://cocl.us/Geospatial_data")

In [61]:
NeighbourhoodLL_table = pd.merge(Neighbourhood_table,Geospatial_CSV,on="Postal Code")
print(NeighbourhoodLL_table.head(11))

   Postal Code           Borough                                 Neighborhood  \
0          M3A        North York                                    Parkwoods   
1          M4A        North York                             Victoria Village   
2          M5A  Downtown Toronto                    Regent Park, Harbourfront   
3          M6A        North York             Lawrence Manor, Lawrence Heights   
4          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
5          M9A         Etobicoke                             Islington Avenue   
6          M1B       Scarborough                               Malvern, Rouge   
7          M3B        North York                                    Don Mills   
8          M4B         East York              Parkview Hill, Woodbine Gardens   
9          M5B  Downtown Toronto                     Garden District, Ryerson   
10         M6B        North York                                    Glencairn   

     Latitude  Longitude  


## Part 3: K-Mean Cluster

Idea:
    1. query the venue for each Postal Code
    2. Summarize venue category by borough (take unique venue id), perform one hot encoding
    3. perform k-mean clustering based on top 10 venues in each borough

In [63]:
#get nearby venue - utility function
CLIENT_ID = 'RBSGZRHE5H03FMTM43053AYPDNC2TJNJO0W50FMFWNVJSHQT' # your Foursquare ID
CLIENT_SECRET = 'VOSJH2GQQSCTKEIWU4HTYIIUQVJ1SDLPYHIXQSAMN42SYSK0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500 #m
LIMIT = 100 #no more than 100 venues per neighbourhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
toronto_venues = getNearbyVenues(names=NeighbourhoodLL_table['Borough'],
                                   latitudes=NeighbourhoodLL_table['Latitude'],
                                   longitudes=NeighbourhoodLL_table['Longitude']
                                  )

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

In [66]:
print(toronto_venues.head())
print(toronto_venues.shape)

  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0   North York              43.753259              -79.329656   
1   North York              43.753259              -79.329656   
2   North York              43.753259              -79.329656   
3   North York              43.725882              -79.315572   
4   North York              43.725882              -79.315572   

                               Venue  Venue Latitude  Venue Longitude  \
0                    Brookbanks Park       43.751976       -79.332140   
1                      Variety Store       43.751974       -79.333114   
2  Corrosion Service Company Limited       43.752432       -79.334661   
3             Victoria Village Arena       43.723481       -79.315635   
4                        Tim Hortons       43.725517       -79.313103   

               Venue Category  
0                        Park  
1           Food & Drink Shop  
2  Construction & Landscaping  
3                Hockey Arena  
4         

In [102]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 263 uniques categories.


In [103]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
onehot_list = list(toronto_onehot.columns)
toronto_onehot['Borough'] = toronto_venues['Neighborhood']

#make sure Neighborhood is the first column
fix_columns = ['Borough'] + onehot_list
toronto_onehot = toronto_onehot[fix_columns]

toronto_onehot.head()

,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
print(toronto_onehot.shape)

(2108, 264)


In [105]:
#for each borough, get the mean of the frequency of occurance of each category
toronto_group = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_group

,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,...,0.000000,0.008621,0.000000,0.000000,0.008621,0.000000,0.000000,0.000000,0.000000,0.008621
1,Downtown Toronto,0.000000,0.000826,0.000826,0.000826,0.000826,0.001652,0.002477,0.001652,0.014038,...,0.000000,0.009909,0.001652,0.000000,0.003303,0.000000,0.004955,0.001652,0.000826,0.005780
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000,0.000000,0.024000
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,...,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.008299,0.000000,0.004149,0.000000,0.000000,0.000000,0.000000,0.000000,0.008299,...,0.000000,0.000000,0.004149,0.004149,0.008299,0.000000,0.000000,0.000000,0.008299,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,...,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.019481,0.000000,0.000000,0.012987,0.000000,0.006494,0.000000,0.000000,0.012987
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000


In [106]:
print(toronto_group.shape)

(10, 264)


In [107]:
#utility function for top x venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
import numpy as np
#print the top 10 most common venues in each borough

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_group['Borough']

for ind in np.arange(toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Restaurant,Sushi Restaurant,Dessert Shop,Clothing Store,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
2,East Toronto,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Ice Cream Shop,Pizza Place,Restaurant,Park,Bakery
3,East York,Coffee Shop,Bank,Sporting Goods Shop,Park,Pharmacy,Pizza Place,Sandwich Place,Burger Joint,Beer Store,Indian Restaurant
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store


In [115]:
#Cluster Neighbourhoods

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_group_clustering = toronto_group.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_group_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 2, 4, 1, 0, 2, 0, 3], dtype=int32)

In [116]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merge = NeighbourhoodLL_table

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merge = toronto_merge.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merge.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Coffee Shop,Clothing Store,Restaurant,Bank,Japanese Restaurant,Pizza Place,Sandwich Place,Park,Grocery Store,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Clothing Store,Restaurant,Bank,Japanese Restaurant,Pizza Place,Sandwich Place,Park,Grocery Store,Fast Food Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Coffee Shop,Clothing Store,Restaurant,Bank,Japanese Restaurant,Pizza Place,Sandwich Place,Park,Grocery Store,Fast Food Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym


In [126]:
#creat map
import folium
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#map centre lat,long
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Borough'], toronto_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [127]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 0, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Coffee Shop,Clothing Store,Restaurant,Bank,Japanese Restaurant,Pizza Place,Sandwich Place,Park,Grocery Store,Fast Food Restaurant
1,North York,0,Coffee Shop,Clothing Store,Restaurant,Bank,Japanese Restaurant,Pizza Place,Sandwich Place,Park,Grocery Store,Fast Food Restaurant
2,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
3,North York,0,Coffee Shop,Clothing Store,Restaurant,Bank,Japanese Restaurant,Pizza Place,Sandwich Place,Park,Grocery Store,Fast Food Restaurant
4,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
...,...,...,...,...,...,...,...,...,...,...,...,...
92,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
96,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
97,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
99,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym


In [128]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 1, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Mississauga,1,Coffee Shop,Hotel,Gym,Intersection,Fried Chicken Joint,Sandwich Place,Middle Eastern Restaurant,American Restaurant,Burrito Place,Mediterranean Restaurant


In [129]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 2, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2,Coffee Shop,Chinese Restaurant,Breakfast Spot,Bank,Fast Food Restaurant,Bakery,Pharmacy,Pizza Place,Lounge,Playground
8,East York,2,Coffee Shop,Bank,Sporting Goods Shop,Park,Pharmacy,Pizza Place,Sandwich Place,Burger Joint,Beer Store,Indian Restaurant
12,Scarborough,2,Coffee Shop,Chinese Restaurant,Breakfast Spot,Bank,Fast Food Restaurant,Bakery,Pharmacy,Pizza Place,Lounge,Playground
14,East York,2,Coffee Shop,Bank,Sporting Goods Shop,Park,Pharmacy,Pizza Place,Sandwich Place,Burger Joint,Beer Store,Indian Restaurant
18,Scarborough,2,Coffee Shop,Chinese Restaurant,Breakfast Spot,Bank,Fast Food Restaurant,Bakery,Pharmacy,Pizza Place,Lounge,Playground
22,Scarborough,2,Coffee Shop,Chinese Restaurant,Breakfast Spot,Bank,Fast Food Restaurant,Bakery,Pharmacy,Pizza Place,Lounge,Playground
23,East York,2,Coffee Shop,Bank,Sporting Goods Shop,Park,Pharmacy,Pizza Place,Sandwich Place,Burger Joint,Beer Store,Indian Restaurant
26,Scarborough,2,Coffee Shop,Chinese Restaurant,Breakfast Spot,Bank,Fast Food Restaurant,Bakery,Pharmacy,Pizza Place,Lounge,Playground
29,East York,2,Coffee Shop,Bank,Sporting Goods Shop,Park,Pharmacy,Pizza Place,Sandwich Place,Burger Joint,Beer Store,Indian Restaurant
32,Scarborough,2,Coffee Shop,Chinese Restaurant,Breakfast Spot,Bank,Fast Food Restaurant,Bakery,Pharmacy,Pizza Place,Lounge,Playground


In [130]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 3, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,York,3,Park,Breakfast Spot,Turkish Restaurant,Discount Store,Sandwich Place,Bus Line,Pool,Hockey Arena,Field,Trail
21,York,3,Park,Breakfast Spot,Turkish Restaurant,Discount Store,Sandwich Place,Bus Line,Pool,Hockey Arena,Field,Trail
56,York,3,Park,Breakfast Spot,Turkish Restaurant,Discount Store,Sandwich Place,Bus Line,Pool,Hockey Arena,Field,Trail
63,York,3,Park,Breakfast Spot,Turkish Restaurant,Discount Store,Sandwich Place,Bus Line,Pool,Hockey Arena,Field,Trail
64,York,3,Park,Breakfast Spot,Turkish Restaurant,Discount Store,Sandwich Place,Bus Line,Pool,Hockey Arena,Field,Trail


In [131]:
toronto_merge.loc[toronto_merge['Cluster Labels'] == 4, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
11,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
17,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
70,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
77,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
88,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
89,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
93,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
94,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store
98,Etobicoke,4,Pizza Place,Sandwich Place,Pharmacy,Coffee Shop,Grocery Store,Gym,Fast Food Restaurant,Café,Fried Chicken Joint,Beer Store


## Observations
- 1st cluster: diverse categories, including clothing shops, japanese/itatlian restaurants
- 2nd cluster: has mediteranean/middle east restaurant, which are not common among other clusters
- 3rd cluster: in addition to coffee shops, this cluster has many chinese restaurants
- 4th cluster: high rank of turkish restaurant is unique to this cluster
- 5th cluster: a lot of pizza place and sandwich place